<a href="https://colab.research.google.com/github/LucaChuang/Newhouse-591-Web-Scraping/blob/main/newhouse591.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver

     |████████████████████████████████| 911kB 8.2MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease

In [ ]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
import warnings
import datetime
import requests
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import csv
import io
import json
import random
warnings.filterwarnings("ignore")

In [ ]:
headers={
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36'
    }

request_url='http://newhouse.591.com.tw/home/housing/info?hid=119282'
res=requests.get(request_url, headers = headers)
bs=BeautifulSoup(res.text,'html.parser')


In [ ]:
# input 建案網址 return 建案建案詳情欄位
# 產出為建案名與12個建案資訊
def getData(url):
    request_url='https://newhouse.591.com.tw/home/housing/info?hid='+str(url).strip()
    headers={
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36'
    }
    res=requests.get(request_url, headers = headers)
    #bs=BeautifulSoup(res.text,'html.parser')

    if res.status_code == 200:
        bs=BeautifulSoup(res.text,'html.parser')
        #先宣告變數為NULL 若無撈到資料則寫入NULL
        title='NULL'
        cost='NULL'
        htype='NULL'
        htype2='NULL'
        htype3='NULL'
        htype4='NULL'
        htype5='NULL'
        htype6='NULL'
        htype7='NULL'
        htype8='NULL'
        htype9='NULL'
        htype10='NULL'
        htype11='NULL'

        # 利用 beautfiulsoup 的 find function 利用 css selector 定位 並撈出指定資料
        title=bs.find('h1').text
        cost=bs.find("dl", {'class':"clearfix"}).findNext("dd").text.strip()
        htype=bs.find("dt", text="建案類別：").findNext("dd").string.strip()
        htype2=bs.find("dt", text="建物形態：").findNext("dd").string.strip().replace(' ', '').replace('\n', '、')
        htype3=bs.find("dt", text="公開銷售：").findNext("dd").string.strip()
        htype4=bs.find("dt", text="基地地址：").findNext("dd").contents[0].string.strip()
        htype5=bs.find("dt", text="交屋屋況：").findNext("dd").string.strip()
        htype6=bs.find("div",{'class':"flex_5 stonefont"}).findNext('span',{'class':''}).text.strip().replace(' ', '')
        htype7=bs.find("dt", text="投資建設：").findNext("dd").string.strip()
        htype8=bs.find("dt", text="營造公司：").findNext("dd").string.strip()
        htype9=bs.find("dt", text="棟戶規劃：").findNext("dd").string.strip()
        htype10=bs.find("dt", text="樓層規劃：").findNext("dd").string.strip()
        htype11=bs.find("dt", text="用途規劃：").findNext("dd").string.strip()
                
        return title,cost,htype, htype2, htype3, htype4, htype5, htype6, htype7, htype8, htype9, htype10, htype11
    else:
        print('link expired:', url)
        return 404, 404, 404, 404, 404, 404, 404

In [ ]:
def get_dynamic_data(url):
    request_url='https://newhouse.591.com.tw/home/housing/dynamic?hid='+str(url).strip()
    headers={
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36'
    }
    res=requests.get(request_url, headers = headers)
    #bs=BeautifulSoup(res.text,'html.parser')

    if res.status_code == 200:
        bs=BeautifulSoup(res.text,'html.parser')
        #先宣告變數為NULL 若無撈到資料則寫入NULL
        news_date = 'NULL'
        news = 'NULL'
        # 利用 beautfiulsoup 的 find function 利用 css selector 定位 並撈出指定資料
        news_date = bs.find("div", {'class':"dynamic-date"}).findNext("span").contents[0].string.strip() + "-" +\
                    bs.find("div", {'class':"dynamic-date"}).findNext("strong").contents[0].string.strip()

        news = bs.find("div", {'class':"dynamic-infolist"}).string
                
        return news_date, news
    else:
        print('link expired:', url)
        return 404, 404, 404, 404, 404, 404, 404

In [ ]:
def main(outputfile, rid, sid, totalpages):
   
   with io.open(outputfile, "w", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["建案名稱", "單價", "建案類別", "建物形態", "公開銷售", "基地地址", "交屋屋況"\
                        ,"格局規劃", "投資建設", "營造公司", "棟戶規劃", "樓層規劃", "用途規劃", "網址"\
                        ,"動態資訊日期日期", "動態資訊"])
        totalpages = totalpages
        print('Total pages: ', totalpages)

        for i in range(1, totalpages+1):
            request_url = "https://newhouse.591.com.tw/home/housing/search?rid="+str(rid)+"&sid="+str(sid)+"&page="+str(i)
            response = requests.get(request_url, headers = headers)
            response = response.json()
            items = response["data"]["items"]

            house_url_list=[] #存放網址list
            for key in items:
                id = key["hid"] # 每個物件的 id
                house_url_list.append(id)
            time.sleep(3)
            #time.sleep(random.randint(5,10))

            # ------------- write into csv ------------- #
            for url in house_url_list:
                title,cost,htype, htype2, htype3, htype4, htype5, htype6, htype7, htype8, htype9, htype10, htype11 = getData(url)
                try:
                  news_date, news = get_dynamic_data(url)
                except:
                  news_date = "Null"
                  news = "Null"
                  print("No news")
                writer.writerow([title,cost,htype, htype2, htype3, htype4, htype5, htype6, htype7, htype8, htype9, htype10, htype11,\
                                 str('https://newhouse.591.com.tw/home/housing/info?hid='+str(url)), news_date, news])
            # ------------------------------------------ #
            print(i/totalpages*100, '%') # print out 完成 %數

In [ ]:
if __name__ == '__main__':
    # -------- configurable parameter -------- #
    # 以台北市不限區舉例(預設網址可能沒寫rid&sid, 點選縣市或區往只會顯示如下)
    # link:https://newhouse.591.com.tw/housing-list.html?rid=1&sid=0

    output_file_name = '/content/drive/MyDrive/output.csv' #設定存放位置與檔名
    rid = 1           # 設定縣市 (台北市 rid = 1)
    sid = 0           # 設定地區 (不限區 sid = 0)
    totalpages = 16    # 設定抓取頁數 # 14頁大概
    # ---------------------------------------- #

    main(output_file_name, rid, sid, totalpages)                                          #匯出csv檔
    read_file = pd.read_csv (output_file_name)
    read_file.to_excel ('/content/drive/MyDrive/output.xlsx', index = None, header=True) #匯出excel檔
    print('\nfinish!')

Total pages:  16
No news
93.75 %
100.0 %

finish!
